In [ ]:
import sys
import types

# Minimal LooseVersion replacement
class LooseVersion:
    def __init__(self, v):
        self.v = v
    def __lt__(self, other):
        return self.v < other.v
    def __eq__(self, other):
        return self.v == other.v
    def __repr__(self):
        return f"LooseVersion({self.v})"

# Patch both 'distutils' and 'setuptools.distutils'
fake_distutils = types.SimpleNamespace(version=types.SimpleNamespace(LooseVersion=LooseVersion))
sys.modules['distutils'] = fake_distutils
sys.modules['setuptools.distutils'] = fake_distutils


import detectron2

from detectron2.data.datasets import register_coco_instances
from detectron2.data import MetadataCatalog
from pycocotools.coco import COCO
register_coco_instances(
    "clps_2020_train", 
    {}, 
    "/mnt/NFSDIR/ARMG-Project-Data/CLPS/images_2020-08-18.json", 
    "/mnt/NFSDIR/ARMG-Project-Data/CLPS/images_2020-08-18"
)


from detectron2.config import get_cfg
from detectron2 import model_zoo
import os

import sys
sys.path.insert(0, "detectron2_repo/projects/PointRend")
import point_rend

cfg = get_cfg()
point_rend.add_pointrend_config(cfg)  # 🔑 This injects keys like MODEL.ROI_MASK_HEAD.FC_DIM
cfg.merge_from_file("detectron2_repo/projects/PointRend/configs/InstanceSegmentation/pointrend_rcnn_R_50_FPN_3x_coco.yaml")
cfg.MODEL.WEIGHTS = "auxfiles/model_final_3c3198.pkl"  #"detectron2://PointRend/InstanceSegmentation/pointrend_rcnn_R_50_FPN_3x_coco/137849600/model_final_3c3198.pkl"
cfg.DATASETS.TRAIN = ("clps_2020_train",)
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 2

coco = COCO("/mnt/NFSDIR/ARMG-Project-Data/CLPS/images_2020-08-18.json")
CLASSES = [cat["name"] for cat in coco.loadCats(coco.getCatIds())]

cfg.SOLVER.IMS_PER_BATCH = 5
cfg.SOLVER.BASE_LR = 0.005
cfg.MODEL.POINT_HEAD.NUM_CLASSES = len(CLASSES)
cfg.MODEL.ROI_HEADS.NUM_CLASSES = len(CLASSES)

cfg.OUTPUT_DIR = "/mnt/NFSDIR/ARMG-Project-Data/CLPS_pointrend_cl"
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)

from pycocotools.coco import COCO
print('\n'.join(CLASSES),file=open(f'{cfg.OUTPUT_DIR}/classes.txt','w'))

In [ ]:
if 0:
    from pycocotools.coco import COCO
    coco = COCO("/mnt/NFSDIR/ARMG-Project-Data/CLPS/images_2020-08-18.json")
    print(coco.getCatIds())
    print(coco.getImgIds())

In [ ]:
from detectron2.engine import DefaultTrainer
from detectron2.utils.events import CommonMetricPrinter, JSONWriter
import os

class TrainerNoTB(DefaultTrainer):
    def build_writers(self):
        return [
            CommonMetricPrinter(self.cfg.SOLVER.MAX_ITER),
            JSONWriter(os.path.join(self.cfg.OUTPUT_DIR, "metrics.json")),
            # TensorboardXWriter is omitted to avoid distutils error
        ]

#from detectron2.engine import DefaultTrainer

trainer = TrainerNoTB(cfg)
trainer.resume_or_load(resume=False)
trainer.train()